# 03 — Equity & Trades Plot
Loads `data/pnl_demo_equity.csv` and `data/pnl_demo_trades.csv`, plots equity & drawdowns, overlays trade exits, plus a monthly return heatmap.


In [ ]:
from pathlib import Path

PROJECT_OVERRIDE = r"C:\Users\speed\Desktop\Forex CFD's system"
ROOT = Path(PROJECT_OVERRIDE)
DATA_1D = ROOT / "data" / "prices_1d"
DATA_1H = ROOT / "data" / "prices_1h"
CFG = ROOT / "config" / "baseline.yaml"
DATA_DIR = DATA_1D if DATA_1D.exists() else DATA_1H

print("Project root:", ROOT)
print("Using data dir:", DATA_DIR)



Project root: C:\Users\speed\Desktop\Forex CFD's system
Using data dir: C:\Users\speed\Desktop\Forex CFD's system\data\prices_1d


In [ ]:
import pandas as pd
import plotly.graph_objects as go

eq_path = ROOT/'data'/'pnl_demo_equity.csv'
tr_path = ROOT/'data'/'pnl_demo_trades.csv'
eq = pd.read_csv(eq_path, parse_dates=['ts']).set_index('ts')

fig = go.Figure()
fig.add_trace(go.Scatter(x=eq.index, y=eq['portfolio_equity'], name='Portfolio'))
for c in eq.columns:
    if c.startswith('equity_'):
        fig.add_trace(go.Scatter(x=eq.index, y=eq[c], name=c))
fig.update_layout(title='Equity Curves', xaxis_title='UTC', yaxis_title='Equity (base=1.0)')
fig.show()

dd = eq['portfolio_equity']/eq['portfolio_equity'].cummax()-1.0
figdd = go.Figure()
figdd.add_trace(go.Scatter(x=dd.index, y=dd.values, name='Drawdown'))
figdd.update_layout(title='Portfolio Drawdown', xaxis_title='UTC', yaxis_title='Drawdown')
figdd.show()


In [ ]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

# Trade exit markers
tr_path = ROOT/'data'/'pnl_demo_trades.csv'
if tr_path.exists():
    tr = pd.read_csv(tr_path, parse_dates=['entry_time','exit_time'])
    exits = tr[['exit_time','symbol','side']].copy()
    exits['y'] = eq['portfolio_equity'].reindex(exits['exit_time']).values
    fig2 = go.Figure()
    fig2.add_trace(go.Scatter(x=eq.index, y=eq['portfolio_equity'], name='Portfolio'))
    fig2.add_trace(go.Scatter(x=exits['exit_time'], y=exits['y'], mode='markers',
                              marker=dict(size=6), name='Trade exits'))
    fig2.update_layout(title='Equity with Trade Exit Markers')
    fig2.show()

# Monthly heatmap
eq_monthly = (1+eq['portfolio_equity'].pct_change().fillna(0)).resample('M').prod()-1
heat = eq_monthly.to_frame('ret').reset_index()
heat['Year']=heat['ts'].dt.year; heat['Month']=heat['ts'].dt.month_name().str[:3]
pivot = heat.pivot(index='Year', columns='Month', values='ret').fillna(0)
months = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
pivot = pivot.reindex(columns=[m for m in months if m in pivot.columns])
px.imshow(pivot, text_auto='.1%', aspect='auto', origin='lower',
          labels=dict(color='Return'), title='Monthly Return Heatmap').show()


C:\Users\speed\AppData\Local\Temp\ipykernel_27948\2021901699.py:17: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.

